In [ ]:
import numpy as np
import pandas as pd
import os
import random
import shutil
import matplotlib.pyplot as plt

from keras.applications import MobileNet
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam, SGD
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import imagenet_utils
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
from IPython.display import Image
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,Input
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image 
from keras.layers.normalization import BatchNormalization
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Model
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.models import load_model

from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import itertools
import shutil
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import os
from keras.preprocessing import image
import cv2
import matplotlib.pyplot as plt
from keras.utils import to_categorical

In [ ]:
#Load data file
data_file = pd.read_csv("/kaggle/input/state-farm-distracted-driver-detection/driver_imgs_list.csv")
data_file

In [ ]:
# Create a new directory
base_dir = 'base_dir'
os.mkdir(base_dir)

#[CREATE FOLDERS INSIDE THE BASE DIRECTORY]

# now we create 2 folders inside 'base_dir':
# train_dir
    # 10 subfolders
# val_dir
    # 10 subfolders

# create a path to 'base_dir' to which we will join the names of the new folders
# train_dir
train_dir = os.path.join(base_dir, 'train_dir')
os.mkdir(train_dir)

# val_dir
val_dir = os.path.join(base_dir, 'val_dir')
os.mkdir(val_dir)

# [CREATE FOLDERS INSIDE THE TRAIN AND VALIDATION FOLDERS]
# Inside each folder we create seperate folders for each class

# create new folders inside train_dir
c0 = os.path.join(train_dir, 'c0')
os.mkdir(c0)
c1 = os.path.join(train_dir, 'c1')
os.mkdir(c1)
c2 = os.path.join(train_dir, 'c2')
os.mkdir(c2)
c3 = os.path.join(train_dir, 'c3')
os.mkdir(c3)
c4 = os.path.join(train_dir, 'c4')
os.mkdir(c4)
c5 = os.path.join(train_dir, 'c5')
os.mkdir(c5)
c6 = os.path.join(train_dir, 'c6')
os.mkdir(c6)
c7 = os.path.join(train_dir, 'c7')
os.mkdir(c7)
c8 = os.path.join(train_dir, 'c8')
os.mkdir(c8)
c9 = os.path.join(train_dir, 'c9')
os.mkdir(c9)


# create new folders inside val_dir
c0 = os.path.join(val_dir, 'c0')
os.mkdir(c0)
c1 = os.path.join(val_dir, 'c1')
os.mkdir(c1)
c2 = os.path.join(val_dir, 'c2')
os.mkdir(c2)
c3 = os.path.join(val_dir, 'c3')
os.mkdir(c3)
c4 = os.path.join(val_dir, 'c4')
os.mkdir(c4)
c5 = os.path.join(val_dir, 'c5')
os.mkdir(c5)
c6 = os.path.join(val_dir, 'c6')
os.mkdir(c6)
c7 = os.path.join(val_dir, 'c7')
os.mkdir(c7)
c8 = os.path.join(val_dir, 'c8')
os.mkdir(c8)
c9 = os.path.join(val_dir, 'c9')
os.mkdir(c9)

In [ ]:
# check that the folders have been created
os.listdir('base_dir/train_dir')


In [ ]:
os.listdir('base_dir/val_dir')

In [ ]:
val_drivers = random.choices(data_file['subject'].unique().tolist(), k=4)
val_drivers

In [ ]:
imgs = list(data_file['img'])

i = 0

for img in imgs:
    i += 1
    label = data_file.loc[data_file['img'] == img, 'classname'].item()
    if data_file.loc[data_file['img'] == img, 'subject'].item() in val_drivers:
        # source path to image
        src = os.path.join('../input/state-farm-distracted-driver-detection/imgs/train', label, img)
        # destination path to image
        dst = os.path.join(val_dir, label, img)
        # copy the image from the source to the destination
        shutil.copyfile(src, dst)
    else:
        # source path to image
        src = os.path.join('../input/state-farm-distracted-driver-detection/imgs/train', label, img)
        # destination path to image
        dst = os.path.join(train_dir, label, img)
        # copy the image from the source to the destination
        shutil.copyfile(src, dst)
        
    if i%1000 == 0:
        print(i)
print(i)

In [ ]:
# check how many train images we have in each folder
val = len(os.listdir('base_dir/train_dir/c0')) + len(os.listdir('base_dir/train_dir/c1')) + len(os.listdir('base_dir/train_dir/c2')) + len(os.listdir('base_dir/train_dir/c3')) + len(os.listdir('base_dir/train_dir/c4')) + len(os.listdir('base_dir/train_dir/c5')) + len(os.listdir('base_dir/train_dir/c6')) + len(os.listdir('base_dir/train_dir/c7')) + len(os.listdir('base_dir/train_dir/c8')) + len(os.listdir('base_dir/train_dir/c9'))

print(len(os.listdir('base_dir/train_dir/c0')) + len(os.listdir('base_dir/train_dir/c1')) + len(os.listdir('base_dir/train_dir/c2')) + len(os.listdir('base_dir/train_dir/c3')) + len(os.listdir('base_dir/train_dir/c4')) + len(os.listdir('base_dir/train_dir/c5')) + len(os.listdir('base_dir/train_dir/c6')) + len(os.listdir('base_dir/train_dir/c7')) + len(os.listdir('base_dir/train_dir/c8')) + len(os.listdir('base_dir/train_dir/c9')))
print(len(os.listdir('base_dir/val_dir/c0')) + len(os.listdir('base_dir/val_dir/c1')) + len(os.listdir('base_dir/val_dir/c2')) + len(os.listdir('base_dir/val_dir/c3')) + len(os.listdir('base_dir/val_dir/c4')) + len(os.listdir('base_dir/val_dir/c5')) + len(os.listdir('base_dir/val_dir/c6')) + len(os.listdir('base_dir/val_dir/c7')) + len(os.listdir('base_dir/val_dir/c8')) + len(os.listdir('base_dir/val_dir/c9')))

In [ ]:
#Load val Images
        
val_image = []
val_label = []

for i in range(10):
    print('Loading images from folder C',i)
    imgs = os.listdir('base_dir/val_dir/c' + str(i))
    for j in range(len(imgs)):

        img_name = 'base_dir/val_dir/c'+str(i)+"/"+imgs[j]
        img = cv2.imread(img_name)
        img = cv2.resize(img, (224,224))
        val_image.append(img)
        val_label.append(i)

In [ ]:
# #Load Images
# import cv2
        
# train_image = []
# image_label = []

# for i in range(10):
#     print('Loading images from folder C',i)
#     imgs = os.listdir('../input/state-farm-distracted-driver-detection/imgs/train/c' + str(i))
#     for j in range(len(imgs)):

#         img_name = '../input/state-farm-distracted-driver-detection/imgs/train/c'+str(i)+"/"+imgs[j]
#         img = cv2.imread(img_name)
#         img = cv2.resize(img, (224,224))
#         label = i
#         driver = data_file[data_file['img'] == imgs[j]]['subject'].values[0]
#         train_image.append([img,label,driver])
#         image_label.append(i)
        

# import random
# random.shuffle(train_image)

In [ ]:
# ## getting list of driver names

# D = []
# for features,labels,drivers in train_image:
#     D.append(drivers)

# ## Deduplicating drivers

# deduped = []

# for i in D:
#     if i not in deduped:
#         deduped.append(i)
    

# ## selecting random drivers for the validation set
# driv_selected = []
# import random
# driv_nums = random.sample(range(len(deduped)), 6)
# for i in driv_nums:
#     driv_selected.append(deduped[i])

In [ ]:
# X_train= []
# y_train = []
# X_test = []
# y_test = []
# D_train = []
# D_test = []

# for features,labels,drivers in train_image:
#     if drivers in driv_selected:
#         X_test.append(features)
#         y_test.append(labels)
#         D_test.append(drivers)
    
#     else:
#         X_train.append(features)
#         y_train.append(labels)
#         D_train.append(drivers)
    
# print (len(X_train),len(X_test))
# print (len(y_train),len(y_test))

In [ ]:
# from keras.utils import to_categorical
# x_train, y_train = np.asarray(X_train), np.asarray(y_train)
# x_test, y_test = np.asarray(X_test), np.asarray(y_test)

# x_train = np.array(x_train, dtype=np.uint8).reshape(-1,224,224,3)
# x_test = np.array(x_test, dtype=np.uint8).reshape(-1,224,224,3)

# # x_train = x_train.astype('float32')
# # x_test = x_test.astype('float32')

# # # normalize to range 0-1
# # x_train /= 255.0
# # x_test /= 255.0

# y_train = to_categorical(y_train)
# y_test = to_categorical(y_test)

In [ ]:
# print(x_train[0], y_train.shape)
# print(x_test.shape, y_test.shape)

# Exploring the dataset

In [ ]:
# print('Unique drivers: ', data_file['subject'].nunique())
# print('Number of images per driver')
# print(data_file['subject'].value_counts())

In [ ]:
# import matplotlib.pyplot as plt
# plt.imshow(cv2.cvtColor(x_train[100], cv2.COLOR_BGR2RGB))
# plt.show()
# print(y_train[100])

# CNN Model

In [ ]:
# from keras.utils import to_categorical
# from keras.models import Sequential
# from keras.layers import *
# from keras.optimizers import RMSprop
# import keras
# from keras.callbacks import EarlyStopping


# model = Sequential()

# size = 16

# model.add(Conv2D(size*2, (3, 3), padding='same', input_shape=(224,224,1)))
# model.add(Activation('elu'))
# model.add(Conv2D(size*2, (3, 3)))
# model.add(Activation('elu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.5))

# model.add(Conv2D(size*4, (3, 3), padding='same'))
# model.add(Activation('elu'))
# model.add(Conv2D(size*4, (3, 3)))
# model.add(Activation('elu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.5))

# model.add(Conv2D(size*8, (3, 3), padding='same'))
# model.add(Activation('elu'))
# model.add(Conv2D(size*8, (3, 3)))
# model.add(Activation('elu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.5))


# model.add(Flatten())
# model.add(Dense(size*16))
# model.add(Activation('elu'))
# model.add(Dropout(0.5))
# model.add(Dense(10))
# model.add(Activation('softmax'))

# # initiate RMSprop optimizer
# opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)

# # Let's train the model using RMSprop
# model.compile(loss='categorical_crossentropy',
#               optimizer=opt,
#               metrics=['accuracy'])


In [ ]:
# model.summary()

In [ ]:
# history = model.fit(x_train, y_train, 
#           validation_data=(x_test, y_test),
#           epochs=20, batch_size=32, callbacks = EarlyStopping(monitor='val_loss', mode='min', patience=5))

In [ ]:
# plt.figure(figsize=(8,8))
# plt.subplot(211)
# plt.title('Cross Entropy Loss')
# plt.plot(history.history['loss'], color='blue', label='train')
# plt.plot(history.history['val_loss'], color='orange', label='test')
# # plot accuracy
# plt.subplot(212)
# plt.title('Classification Accuracy')
# plt.plot(history.history['accuracy'], color='blue', label='train')
# plt.plot(history.history['val_accuracy'], color='orange', label='test')

In [ ]:
# from keras.models import load_model

# model.save('CNN_SF_distracted_drivers_2.h5')

In [ ]:
# _, acc = model.evaluate(x_test, y_test, verbose=10)
# print(acc)

# **MobileNet model**

In [ ]:
# base_model=MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

# final_layer = base_model.output
# final_layer = GlobalAveragePooling2D()(final_layer)

# pred=Dense(10,activation='softmax')(final_layer) #final layer with softmax activation

# model = Model(inputs=base_model.input, outputs=pred)

# model.summary()

In [ ]:
# opt = optimizers.SGD(lr = 0.005)
# # opt = optimizers.Adam(lr=0.001)


# model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy']) 

In [ ]:
# filepath = "mobilenet_sgd0.001_4_v2.h5"
# checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
# earlystopper = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

# datagen_val = ImageDataGenerator()

# datagen_train = ImageDataGenerator(height_shift_range=0.5,width_shift_range = 0.5,zoom_range = 0.5,rotation_range=30)

# train_path = 'base_dir/train_dir'
# valid_path = 'base_dir/val_dir'

# IMAGE_SIZE = 224

# train_gen = datagen_train.flow_from_directory(train_path,
#                                         target_size=(IMAGE_SIZE,IMAGE_SIZE),
#                                         batch_size=64,
#                                         class_mode='categorical')

# val_gen = datagen_val.flow_from_directory(valid_path,
#                                         target_size=(IMAGE_SIZE,IMAGE_SIZE),
#                                         batch_size=64,
#                                         class_mode='categorical')

# #datagen.fit(X_train)
# # data_generator = datagen.flow(x_train, y_train, batch_size = 4)

# # Fits the model on batches with real-time data augmentation:
# mobilenet_model = model.fit_generator(train_gen,steps_per_epoch = val / 64, callbacks=[checkpoint,earlystopper],
#                         epochs = 25, verbose = 1, validation_data = val_gen)

In [ ]:
# plt.figure(figsize=(8,8))
# plt.subplot(211)
# plt.title('Cross Entropy Loss')
# plt.plot(mobilenet_model.history['loss'], color='blue', label='train')
# plt.plot(mobilenet_model.history['val_loss'], color='orange', label='test')
# # plt.ylim(0,1)
# # plot accuracy
# plt.subplot(212)
# plt.title('Classification Accuracy')
# plt.plot(mobilenet_model.history['accuracy'], color='blue', label='train')
# plt.plot(mobilenet_model.history['val_accuracy'], color='orange', label='test')
# plt.ylim(0,1)

In [ ]:
# #delete base_dir

# shutil.rmtree('base_dir')

In [ ]:
# val_image = np.array(val_image, dtype=np.uint8).reshape(-1,224,224,3)

# val_label = to_categorical(val_label)

In [ ]:

# predictions = model.predict(val_image, steps=len(val_image), verbose=1)

In [ ]:
# pred = predictions.argmax(axis=1)
# pred_val = val_label.argmax(axis=1)

In [ ]:
# cm = confusion_matrix(pred_val, pred)
# print(cm)

# VGG16 Model

In [ ]:
from keras.layers import Input
vgg16_input = Input(shape = (224, 224, 3), name = 'Image_input')


## The VGG model

from keras.applications.vgg16 import VGG16, preprocess_input

#Get back the convolutional part of a VGG network trained on ImageNet
model_vgg16_conv = VGG16(weights='imagenet', include_top=False, input_tensor = vgg16_input)
model_vgg16_conv.summary()

In [ ]:
#Use the generated model 

output_vgg16_conv = model_vgg16_conv(vgg16_input)

#Add the fully-connected layers 

x = Flatten(name='flatten')(output_vgg16_conv)

x = Dense(10, activation='softmax', name='predictions')(x)

vgg16_pretrained = Model(inputs = vgg16_input, outputs = x)
vgg16_pretrained.summary()

# Compile CNN model
sgd = optimizers.SGD(lr = 0.005)
vgg16_pretrained.compile(loss='categorical_crossentropy',optimizer = sgd,metrics=['accuracy'])

In [ ]:
datagen_val = ImageDataGenerator(height_shift_range=0.5,width_shift_range = 0.5,zoom_range = 0.5,rotation_range=30,rescale=1./255)

datagen_train = ImageDataGenerator(height_shift_range=0.5,width_shift_range = 0.5,zoom_range = 0.5,rotation_range=30, rescale=1./255)

train_path = 'base_dir/train_dir'
valid_path = 'base_dir/val_dir'

IMAGE_SIZE = 224

train_gen = datagen_train.flow_from_directory(train_path,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=64,
                                        class_mode='categorical')

val_gen = datagen_val.flow_from_directory(valid_path,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=64,
                                        class_mode='categorical')

In [ ]:
checkpoint = ModelCheckpoint("vgg16_sgd0.001_4_{epoch:02d}_{val_accuracy:.2f}.h5", monitor='val_accuracy', verbose=1, mode='max', save_freq='epoch')
earlystopper = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

# Fits the model on batches with real-time data augmentation:
vgg16_model = vgg16_pretrained.fit(train_gen,steps_per_epoch = val / 64, callbacks=[checkpoint,earlystopper],
                        epochs = 25, verbose = 1, validation_data = val_gen)

In [ ]:
plt.figure(figsize=(8,8))
plt.subplot(211)
plt.title('Cross Entropy Loss')
plt.plot(vgg16_model.history['loss'], color='blue', label='train')
plt.plot(vgg16_model.history['val_loss'], color='orange', label='test')
# plt.ylim(0,1)
# plot accuracy
plt.subplot(212)
plt.title('Classification Accuracy')
plt.plot(vgg16_model.history['accuracy'], color='blue', label='train')
plt.plot(vgg16_model.history['val_accuracy'], color='orange', label='test')
plt.ylim(0,1)

In [ ]:
#delete base_dir

shutil.rmtree('base_dir')

In [ ]:
val_image = np.array(val_image, dtype=np.uint8).reshape(-1,224,224,3)
val_label = to_categorical(val_label)
print(val_label)

In [ ]:
val_image = val_image.astype('float32')

In [ ]:
pred_valid = val_label.argmax(axis=1)
cm1 = confusion_matrix(pred_valid, pred_valid)
print(cm1)

In [ ]:
# model = load_model('../input/state-farm-distracted-drivers-detection/vgg16_sgd0.001_4_10_0.79.h5')

predictions = vgg16_pretrained.predict(val_image, steps=len(val_image), verbose=1)

pred = predictions.argmax(axis=1)
pred_val = val_label.argmax(axis=1)

cm = confusion_matrix(pred_val, pred)
print(cm)

In [ ]:
# test_data_dir = r'../input/state-farm-distracted-driver-detection/imgs/test/'

# class_labels = [
#     "normal driving",
#     "texting - right",
#     "talking on the phone - right",
#     "texting - left",
#     "talking on the phone - left",
#     "operating the radio",
#     "drinking",
#     "reaching behind",
#     "hair and makeup",
#     "talking to passenger"]

# file_names = np.random.choice(os.listdir(test_data_dir),50)

# img_arrays = []

# for file_name in file_names:
#     img = image.load_img(os.path.join(test_data_dir, file_name), target_size=(224, 224))
#     img_array = image.img_to_array(img)
#     img_arrays.append(img_array)

# img_arrays = np.array(img_arrays)
# # img_arrays = img_arrays.astype('float32')
# predictions = model.predict(np.array(img_arrays).reshape(-1,224,224,3))
# # predictions = model.predict(img_arrays)


# label_indexes = np.argmax(predictions, axis=1)
# probabilities = np.max(predictions, axis=1)

# for (file_name, label_index, probability) in zip(file_names, label_indexes, probabilities):

#     label_with_probability = "{}: {:.2f}%".format(class_labels[label_index], probability * 100)

#     import cv2

#     image = cv2.imread(os.path.join(test_data_dir, file_name))

#     cv2.putText(image, label_with_probability.upper(), (100, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
    
#     plt.imshow(image)
#     plt.show()